## Introduction

In this notebook, we will be adding a few improvements - packed padded sequences and masking - to the model from the previous notebook. Packed padded sequences are used to tell our RNN to skip over padding tokens in our encoder. Masking explicitly forces the model to ignore certain values, such as attention over padded elements. Both of these techniques are commonly used in NLP. 

We will also look at how to use our model for inference, by giving it a sentence, seeing what it translates it as and seeing where exactly it pays attention to when translating each word.

Finally, we will use the BLEU metric to measure the quality of our translations.

## Preparing Data

First, we will import all the modules as before, with the addition of the `matplotlib` modules used for viewing the attention.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

from tqdm import tqdm

Next, we'll set the random seed for reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

As before, we'll import spaCy and define the German and English tokenizers.

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

When using packed padded sequences, we need to tell PyTorch how long the actual (non-padded) sequences are. Luckily for us, TorchText's `Field` objects allow us to use the `include_lengths` argument, this will cause our `batch.src` to be a tuple. The first element of the tuple is the same as before, a batch of numericalized source sentence as a tensor, and the second element is the non-padded lengths of each source sentence within the batch.

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

We then load the data.

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

And build the vocabulary.

In [7]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

Next, we handle the iterators.

One quirk about packed padded sequences is that all elements in the batch need to be sorted by their non-padded lengths in descending order, i.e. the first sentence in the batch needs to be the longest. We use two arguments of the iterator to handle this, `sort_within_batch` which tells the iterator that the contents of the batch need to be sorted, and `sort_key` a function which tells the iterator how to sort the elements in the batch. Here, we sort by the length of the `src` sentence.

In [8]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

## Building the Model

### Encoder

Next up, we define the encoder.

The changes here all within the `forward` method. It now accepts the lengths of the source sentences as well as the sentences themselves. 

After the source sentence (padded automatically within the iterator) has been embedded, we can then use `pack_padded_sequence` on it with the lengths of the sentences. Note that the tensor containing the lengths of the sequences must be a CPU tensor as of the latest version of PyTorch, which we explicitly do so with `to('cpu')`. `packed_embedded` will then be our packed padded sequence. This can be then fed to our RNN as normal which will return `packed_outputs`, a packed tensor containing all of the hidden states from the sequence, and `hidden` which is simply the final hidden state from our sequence. `hidden` is a standard tensor and not packed in any way, the only difference is that as the input was a packed sequence, this tensor is from the final **non-padded element** in the sequence.

We then unpack our `packed_outputs` using `pad_packed_sequence` which returns the `outputs` and the lengths of each, which we don't need. 

The first dimension of `outputs` is the padded sequence lengths however due to using a packed padded sequence the values of tensors when a padding token was the input will be all zeros.

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.input_dim = input_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        # src = [src len, batch size]
        # src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        # embedded = [src len, batch size, emb dim]
                
        # need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        # packed_outputs is a packed sequence containing all hidden states
        # hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        # outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        # outputs = [src len, batch size, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        # hidden [-2, :, : ] is the last of the forwards RNN 
        # hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        # outputs = [src len, batch size, enc hid dim * 2]
        # hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

The attention module is where we calculate the attention values over the source sentence. 

Previously, we allowed this module to "pay attention" to padding tokens within the source sentence. However, using *masking*, we can force the attention to only be over non-padding elements.

The `forward` method now takes a `mask` input. This is a **[batch size, source sentence length]** tensor that is 1 when the source sentence token is not a padding token, and 0 when it is a padding token. For example, if the source sentence is: ["hello", "how", "are", "you", "?", `<pad>`, `<pad>`], then the mask would be [1, 1, 1, 1, 1, 0, 0].

We apply the mask after the attention has been calculated, but before it has been normalized by the `softmax` function. It is applied using `masked_fill`. This fills the tensor at each element where the first argument (`mask == 0`) is true, with the value given by the second argument (`-1e10`). In other words, it will take the un-normalized attention values, and change the attention values over padded elements to be `-1e10`. As these numbers will be miniscule compared to the other values they will become zero when passed through the `softmax` layer, ensuring no attention is payed to padding tokens in the source sentence.

In [10]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        # hidden = [batch size, dec hid dim]
        # encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        # repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        # hidden = [batch size, src len, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        # energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        # attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

### Decoder

The decoder only needs a few small changes. It needs to accept a mask over the source sentence and pass this to the attention module. As we want to view the values of attention during inference, we also return the attention tensor.

In [11]:
# Based on the `step` function from adamklec's CopyNet implementation: https://github.com/adamklec/copynet/blob/master/model/copynet_decoder.py

from torch.autograd import Variable
from utils import to_one_hot

class Decoder(nn.Module):
    def __init__(self,input_dim, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        # used for copy mechanism
        self.copy_W = nn.Linear(enc_hid_dim, enc_hid_dim * 2)
        
        # self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        self.out = nn.Linear(dec_hid_dim, output_dim)

        
    def forward(self, input, hidden, encoder_outputs, mask, one_hot_input_seq):
             
        # input = [batch size]
        # hidden = [batch size, dec hid dim]
        # encoder_outputs = [src len, batch size, enc hid dim * 2]
        # mask = [batch size, src len]
        
        # `prev_idx` in adamklec's implementation
        input = input.unsqueeze(0)
        
        # input = [1, batch size]
        
        # line 94 of adamklec's implementation
        embedded = self.dropout(self.embedding(input)) # embed input (i.e. previous output token)
        
        # embedded = [1, batch size, emb dim]
        
        # `attn_weights` in adamklec's implementation
        a = self.attention(hidden, encoder_outputs, mask)
                
        # a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        # a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        # encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        # `context` in adamklec's implementation
        weighted = torch.bmm(a, encoder_outputs)
        
        # weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        # weighted = [1, batch size, enc hid dim * 2]
        
        # line 96 in adamklec's implementation
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        # rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        # line 103 in adamklec's implementation
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        # output = [seq len, batch size, dec hid dim * n directions]
        # hidden = [n layers * n directions, batch size, dec hid dim]
        
        # seq len, n layers and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, dec hid dim]
        # hidden = [1, batch size, dec hid dim]
        # print('shape of output:', output.shape)
        # print('shape of hidden:', hidden.shape)
        # this also means that output == hidden
        assert (output == hidden).all()

        # --- Copy mechanism taken from adamklec's implementation ---
        batch_size = encoder_outputs.shape[0]
        hidden_size = hidden.shape[2]
        transformed_hidden = self.copy_W(output).view(batch_size, hidden_size * 2, 1)
        # dim of `transformed_hidden` = [batch size, hidden_size * 2, 1]

        # dim of `encoder_outputs` = [batch size, src len, enc hid dim * 2]
        # the following is linear. add activation function before multiplying.
        copy_score_seq = torch.bmm(encoder_outputs, transformed_hidden)
        # dim of `copy_scores_seq` = [batch size, seq_length, 1]
        # print('shape of copy_score_seq:', copy_score_seq.shape)

        # Suspicion: `len(self.lang.tok_to_idx)` is `output_dim`
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        seq_length = encoder_outputs.shape[1]

        # one_hot_input_seq = to_one_hot(src, self.output_dim + seq_length).to(device)
        # dim of `one_hot_input_seq` = [batch size, vocab_size + seq_length]
        # print('shape of copy_score_seq transposed:', torch.transpose(copy_score_seq, 1, 2).shape)
        # print('shape of one_hot_input_seq:', one_hot_input_seq.shape)

        # dim of copy_scores: [b, input_vocab_size + seq_length]
        copy_scores = torch.bmm(torch.transpose(copy_score_seq, 1, 2), one_hot_input_seq).squeeze(1).to(device)

        # tokens not present in the input sequence
        missing_token_mask = (one_hot_input_seq.sum(dim=1) == 0)

        # print(missing_token_mask)

        # missing tokens are not part of any sequence
        # missing_token_mask[:, 0] = 1
        copy_scores = copy_scores.masked_fill(missing_token_mask, -1000000.0)
        # print('shape of copy_scores', copy_scores.shape)
        # print('copy_scores', copy_scores)

        # --- Generate mechanism taken from adamklec's implementation ---
        # [b, vocab size]
        # Suspicion: might need to change the dimension for this squeeze
        gen_scores = self.out(output.squeeze(0)).to(device)
        # embedded = embedded.squeeze(0)
        # output = output.squeeze(0)
        # weighted = weighted.squeeze(0)
        # gen_scores = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        # penalize <MSK> tokens in generate mode too
        # gen_scores[:, 0] = -1000000.0
        # print('shape of gen_scores', gen_scores.shape)

        # Combine results from copy and generate mechanisms
        combined_scores = torch.cat((gen_scores, copy_scores), dim=1).to(device)
        probs = F.softmax(combined_scores, dim=1).to(device)
        # print('shape of probs:', probs.shape)
        gen_probs = probs[:, :self.output_dim]
        # print('shape of gen_probs:', gen_probs.shape)

        # gen_padding = Variable(torch.zeros(batch_size, seq_length)).to(device)
        # gen_probs = torch.cat((gen_probs, gen_padding), dim=1).to(device)
        # print('shape of prediction:', prediction.shape)
        # dim of gen_probs: [b, vocab_size + seq_length]
        
        copy_probs = probs[:, self.output_dim:]
        # print('shape of gen_probs:', gen_probs.shape)
        # print('shape of copy_probs:', copy_probs.shape)
        prediction = torch.cat((gen_probs, copy_probs), 1)
        
        # prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        # dim of prediction: [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

The overarching seq2seq model also needs a few changes for packed padded sequences, masking and inference. 

We need to tell it what the indexes are for the pad token and also pass the source sentence lengths as input to the `forward` method.

We use the pad token index to create the masks, by creating a mask tensor that is 1 wherever the source sentence is not equal to the pad token. This is all done within the `create_mask` function.

The sequence lengths as needed to pass to the encoder to use packed padded sequences.

The attention at each time-step is stored in the `attentions` 

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, input_dim, src_pad_idx, device):
        super().__init__()
        
        self.input_dim = input_dim
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        # src = [src len, batch size]
        # src_len = [batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]

        # print('shape of src[0]:', src.shape[0])
        # print('shape of trg[0]:', trg.shape[0])

        trg_vocab_size = self.decoder.output_dim
        src_vocab_size = self.encoder.input_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size + src_vocab_size + src.shape[0]).to(self.device)
        
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        # first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)
        
        # print('shape of src:', src.shape)
        # print('shape of input:', input.shape)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        src = src.permute(1, 0)
        one_hot_input_seq = to_one_hot(src, self.input_dim + src.shape[1]).to(device)

        # mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            # insert input token embedding, previous hidden state, all encoder hidden states 
            #   and mask
            # receive output tensor (predictions) and new hidden state
            # extra input: src
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask, one_hot_input_seq)
            
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            # get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

## Training the Seq2Seq Model

Next up, initializing the model and placing it on the GPU.

In [13]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(INPUT_DIM, OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, INPUT_DIM, SRC_PAD_IDX, device).to(device)

Then, we initialize the model parameters.

In [14]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (copy_W): Linear(in_features=512, out_features=1024, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (out): Linear(in_features=512, out_features=5893, bias=True)
  )
)

We'll print out the number of trainable parameters in the model, noticing that it has the exact same amount of parameters as the model without these improvements.

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,500,677 trainable parameters


Then we define our optimizer and criterion. 

The `ignore_index` for the criterion needs to be the index of the pad token for the target language, not the source language.

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [17]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Next, we'll define our training and evaluation loops.

As we are using `include_lengths = True` for our source field, `batch.src` is now a tuple with the first element being the numericalized tensor representing the sentence and the second element being the lengths of each sentence within the batch.

Our model also returns the attention vectors over the batch of source source sentences for each decoding time-step. We won't use these during the training/evaluation, but we will later for inference.

In [18]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator)):
        
        src, src_len = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        # print(output.shape)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Then, we'll define a useful function for timing how long epochs take.

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

The penultimate step is to train our model. Notice how it takes almost half the time as our model without the improvements added in this notebook.

In [21]:
N_EPOCHS = 10
CLIP = 1

my_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(my_device)

best_valid_loss = float('inf')

# Benson: list to store training loss and validation
train_loss_list = []
valid_loss_list = []

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    # Benson: extra code for plotting purposes
    train_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)

# Benson: saving the data in case matplotlib code does not work
np.save('train_loss_list_new.npy', np.array(train_loss_list))
np.save('valid_loss_list_new.npy', np.array(valid_loss_list))


0it [00:00, ?it/s]cuda
3it [00:00,  5.29it/s]


RuntimeError: CUDA error: device-side assert triggered

Benson: Plot the training and validation loss curves.

In [ ]:
# Benson: code for plotting
x_axis = np.arange(1, len(train_loss_list) + 1, 1)
plt.figure()
plt.plot(x_axis, train_loss_list, label='Training Loss')
plt.plot(x_axis, valid_loss_list, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss per Epoch')
plt.savefig('loss_per_epoch.jpg')
plt.show()
plt.clf()
plt.close()

Finally, we load the parameters from our best validation loss and get our results on the test set.

We get the improved test perplexity whilst almost being twice as fast!

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

## Inference

Now we can use our trained model to generate translations.

**Note:** these translations will be poor compared to examples shown in paper as they use hidden dimension sizes of 1000 and train for 4 days! They have been cherry picked in order to show off what attention should look like on a sufficiently sized model.

Our `translate_sentence` will do the following:
- ensure our model is in evaluation mode, which it should always be for inference
- tokenize the source sentence if it has not been tokenized (is a string)
- numericalize the source sentence
- convert it to a tensor and add a batch dimension
- get the length of the source sentence and convert to a tensor
- feed the source sentence into the encoder
- create the mask for the source sentence
- create a list to hold the output sentence, initialized with an `<sos>` token
- create a tensor to hold the attention values
- while we have not hit a maximum length
  - get the input tensor, which should be either `<sos>` or the last predicted token
  - feed the input, all encoder outputs, hidden state and mask into the decoder
  - store attention values
  - get the predicted next token
  - add prediction to current output sentence prediction
  - break if the prediction was an `<eos>` token
- convert the output sentence from indexes to tokens
- return the output sentence (with the `<sos>` token removed) and the attention values over the sequence

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
        
        # greedy search - taking the argmax of the output
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

Benson: Beam search implementation for translation task

In [ ]:
def translate_sentence_beam(sentence, src_field, trg_field, model, device, max_len = 50, num_beams = 5, print_output = False):
    # following https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
    # attention is currently left out in this implementation of the translation task.

    model.eval()

    # number of beams (hyperparameter)
    K = num_beams

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    # trg_indexes = [[trg_field.vocab.stoi[trg_field.init_token]]]

    # attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)

    # order of items per element: token idx list, LLH score, hidden state
    sequences = [[[trg_field.vocab.stoi[trg_field.init_token]], 0.0, hidden]]

    # token idx list, score
    final_sequences = []
    
    # loop for max length
    for i in range(max_len):

        # early stopping condition
        # if we already have K results ending with EOS, then we should stop translating
        if len(final_sequences) == K:
            break

        # order of items: token idx list, LLH score, output, hidden
        collective_output = list()

        # for-loop to do one trg_indexes at a time
        for j in range(len(sequences)):

            # early stopping condition
            # if we have a list that already contains EOS, we should decrease K by 1, and keep track of that list for a contender
            if sequences[j][0][-1] == trg_field.vocab.stoi[trg_field.eos_token]:
                final_sequences.append([sequences[j][0], sequences[j][1]])
                K -= 1
                continue

            trg_tensor = torch.LongTensor([sequences[j][0][-1]]).to(device)
            hidden_input = sequences[j][2]
            with torch.no_grad():
                # output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)
                output, hidden_output, _ = model.decoder(trg_tensor, hidden_input, encoder_outputs, mask)

            # collecting all the output into one list
            collective_output.append([sequences[j][0], sequences[j][1], output, hidden_output])

        # attentions[i] = attention
    
        all_candidates = list()
        all_eos = True
        # expand each current candidate
        for j in range(len(collective_output)):
            seq, score, output_tensor, hidden_for_seq = collective_output[j]
            # if the seq already ends with EOS, then we should skip it
            if len(seq) > 1 and seq[-1] == trg_field.vocab.stoi[trg_field.eos_token]:
                continue
            else:
                all_eos = False
            # Softmax so we can retrieve probabilities
            m = nn.Softmax(dim=0)
            row = m(output_tensor.squeeze(0))
            # grab top K sequences out of row
            top_K_rows, top_K_indices = torch.topk(row, K)
            top_K_rows, top_K_indices = top_K_rows.tolist(), top_K_indices.tolist()
            # for each output, append to its sequence as well as updating its score
            # NOTE: we are not penalizing longer sequences, so early stopping is implemented to prevent long sequences
            for k in range(len(top_K_indices)):
                # instead of subtracting the log, we add the log to our scores (making our scores negative)
                candidate = [seq + [top_K_indices[k]], score + np.log(top_K_rows[k]), hidden_for_seq]
                all_candidates.append(candidate)
        
        # return early if all sequences are true
        if all_eos == True:
            break

        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])

        # select K best (which is the greatest K scores, or the K least negative scores)
        sequences = ordered[-K:]

    best_score = None
    best_trg_tokens = []
    for idx in range(len(final_sequences)):
        trg_tokens = [trg_field.vocab.itos[i] for i in final_sequences[idx][0]]
        # print out results for user consumption
        if print_output is True:
            print('score: ', final_sequences[idx][1],'\n\ttranslation: ', trg_tokens)
        if best_score is None or final_sequences[idx][1] > best_score:
            best_score = final_sequences[idx][1]
            best_trg_tokens = trg_tokens

    # returning -1 as attention for now (unused)
    return best_trg_tokens[1:], -1
    

Next, we'll make a function that displays the model's attention over the source sentence for each target token generated.

In [ ]:
def display_attention(file_name, sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    
    x_ticks = [''] + ['<sos>'] + [t.lower() for t in sentence] + ['<eos>']
    y_ticks = [''] + translation
     
    ax.set_xticklabels(x_ticks, rotation=45)
    ax.set_yticklabels(y_ticks)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.savefig(file_name + '.jpg')
    plt.show()
    plt.close()

Now, we'll grab some translations from our dataset and see how well our model did. Note, we're going to cherry pick examples here so it gives us something interesting to look at, but feel free to change the `example_idx` value to look at different examples.

First, we'll get a source and target from our dataset.

Benson: I set `example_idx` to be 30.

In [ ]:
example_idx = 30

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

Then we'll use our `translate_sentence` function to get our predicted translation and attention. We show this graphically by having the source sentence on the x-axis and the predicted translation on the y-axis. The lighter the square at the intersection between two words, the more attention the model gave to that source word when translating that target word.

Below is an example the model attempted to translate, it gets the translation correct except changes *are fighting* to just *fighting*.

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device) 

beam_translation, _ = translate_sentence_beam(src, SRC, TRG, model, device, print_output=True)

print(f'\npredicted greedy trg = {translation}')
print(f'\npredicted beam trg = {beam_translation}')

In [ ]:
display_attention('first_sent', src, translation, attention)

Translations from the training set could simply be memorized by the model. So it's only fair we look at translations from the validation and testing set too.

Starting with the validation set, let's get an example.

I set `example_idx` to be 21.

In [ ]:
example_idx = 21

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

Then let's generate our translation and view the attention.

Here, we can see the translation is the same except for swapping *female* with *woman*.

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

beam_translation, _ = translate_sentence_beam(src, SRC, TRG, model, device, print_output=True)

print(f'\npredicted trg = {translation}')
print(f'\npredicted beam trg = {beam_translation}')

display_attention('second_sent', src, translation, attention)

Finally, let's get an example from the test set.

Benson: I set `example_idx` to 9.

In [ ]:
example_idx = 9

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

Again, it produces a slightly different translation than target, a more literal version of the source sentence. It swaps *mountain climbing* for *climbing a mountain*.

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

beam_translation, _ = translate_sentence_beam(src, SRC, TRG, model, device, print_output=True)

print(f'\npredicted trg = {translation}')
print(f'\npredicted beam trg = {beam_translation}')

display_attention('third_sent', src, translation, attention)

## BLEU

Previously we have only cared about the loss/perplexity of the model. However there metrics that are specifically designed for measuring the quality of a translation - the most popular is *BLEU*. Without going into too much detail, BLEU looks at the overlap in the predicted and actual target sequences in terms of their n-grams. It will give us a number between 0 and 1 for each sequence, where 1 means there is perfect overlap, i.e. a perfect translation, although is usually shown between 0 and 100. BLEU was designed for multiple candidate translations per source sequence, however in this dataset we only have one candidate per source.

We define a `calculate_bleu` function which calculates the BLEU score over a provided TorchText dataset. This function creates a corpus of the actual and predicted translation for each source sentence and then calculates the BLEU score.

Benson: Added functionality to translate sentence with beam search.

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, file_name, max_len = 50, beam = False):
    
    trgs = []
    pred_trgs = []
    
    for datum in tqdm(data):
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        if beam is False:
            pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        else:
            pred_trg, _ = translate_sentence_beam(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
    
    np.save(file_name + '_pred_list.npy', np.array(pred_trgs))
    np.save(file_name + '_trg_list.npy', np.array(trgs))
        
    return bleu_score(pred_trgs, trgs)

We get a BLEU of around 28. If we compare it to the paper that the attention model is attempting to replicate, they achieve a BLEU score of 26.75. This is similar to our score, however they are using a completely different dataset and their model size is much larger - 1000 hidden dimensions which takes 4 days to train! - so we cannot really compare against that either.

This number isn't really interpretable, we can't really say much about it. The most useful part of a BLEU score is that it can be used to compare different models on the same dataset, where the one with the **higher** BLEU score is "better".

In [ ]:
bleu = calculate_bleu(test_data, SRC, TRG, model, device, 'bleu')

# pred_trgs = np.load('bleu_pred_list.npy', allow_pickle=True)
# trgs = np.load('bleu_trg_list.npy', allow_pickle=True)
# bleu = bleu_score(pred_trgs, trgs)

print(f'Greedy BLEU score = {bleu*100:.2f}')

# This code might take a while as it is not optimized for GPU usage
beam_bleu = calculate_bleu(test_data, SRC, TRG, model, device, 'beam_bleu', beam=True)

# beam_pred_trgs = np.load('beam_bleu_pred_list.npy', allow_pickle=True)
# beam_trgs = np.load('beam_bleu_trg_list.npy', allow_pickle=True)
# beam_bleu = bleu_score(beam_pred_trgs, beam_trgs)

print(f'Beam BLEU score = {beam_bleu*100:.2f}')

In the next tutorials we will be moving away from using recurrent neural networks and start looking at other ways to construct sequence-to-sequence models. Specifically, in the next tutorial we will be using convolutional neural networks.